<a href="https://colab.research.google.com/github/paridhika/Cloudlab-RDMA/blob/main/Resnet50_n_params.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# According to PyTorch

In [ ]:
from collections import Counter

from torchvision.models import resnet50
import torch
from torchsummary import summary

In [ ]:
summary(resnet50(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

# According to TF

In [1]:
import tensorflow as tf

In [ ]:
tf.keras.applications.resnet.ResNet50(weights=None).summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

# In summary

In [16]:
import tensorflow as tf
model = tf.keras.applications.resnet.ResNet50(weights=None);
# model.summary();
len(model.layers)
trainableLayerNameList = [layer.name for layer in model.layers if layer.trainable_variables]
print(len(trainableLayerNameList));

107


In [ ]:
allreduce_layers = 0

# iterate over the layers in the model
for layer in model.layers:
    # check if the layer has trainable parameters
    if layer.trainable_variables:
        # if the layer has trainable parameters, increment the counter
        allreduce_layers += 1

# print the number of layers involved in the AllReduce operation
print(f'Number of layers involved in AllReduce: {allreduce_layers}')

In [17]:
allreduce_layers = 0;
for layer in model.layers:
    # check if the layer has trainable parameters
    if not layer.trainable_variables:
        # if the layer has trainable parameters, increment the counter
        if "add" in layer.name:
          allreduce_layers += 1
          print(layer.name);
# print the number of layers involved in the AllReduce operation
print(f'Number of layers not involved in AllReduce: {allreduce_layers}')

conv2_block1_add
conv2_block2_add
conv2_block3_add
conv3_block1_add
conv3_block2_add
conv3_block3_add
conv3_block4_add
conv4_block1_add
conv4_block2_add
conv4_block3_add
conv4_block4_add
conv4_block5_add
conv4_block6_add
conv5_block1_add
conv5_block2_add
conv5_block3_add
Number of layers not involved in AllReduce: 16
